<img alt="boost-histogram logo" src="images/BoostHistogramPythonLogo.png" style="width:40%;"></img>

# Python Histogramming

### Henry Schreiner

Setup (done for you on binder):

```bash
conda env create
conda activate bh-talk
jupyter lab
```

The interesting parts of the environment:

In [ ]:
from importlib import metadata
packages = "boost-histogram hist uhi mplhep histoprint".split()
for package in packages:
    meta = metadata.metadata(package)
    print(f"{package:16} {meta['version']}   {meta['Summary']}")

> #### Note:
>
> boost-histogram and the core Hist functionality has minimal, lightweight requirements and works almost anywhere. Anything beyond that is for plotting or other speical functionality.

In [ ]:
import hist
from hist import Hist
import numpy as np
import matplotlib.pyplot as plt
import mplhep
from PIL import Image
import pandas as pd

> #### IPython setup
> 
> For the demos, let's see the last expression so it's easier to follow what's happening, even if it is an assignment. The default here is 'last_expr', but 'last_expr_or_assign' is better for demos. Also see 'last', 'all', and 'none'.

In [ ]:
%config InteractiveShell.ast_node_interactivity="last_expr_or_assign"


# What is boost-histogram / Hist?

The Python ecosystem is missing a good Histogram object. NumPy can perform a histogram operation, but it does not produce an object, and there are limitations and performance issues. The closest thing we have to a histogram is in ROOT, a massive HEP C++ framework with JIT bindings.

We have introduced:

* boost-histogram: High performance composable histograms using Boost.Histogram (header-only Boost library)
* hist: Extends boost-histogram with new features and shortcuts, REPL/analysis friendly
* UHI: Unified Histogram Interface, describes Protocol and common features

Our other libraries (`uproot`, for reading ROOT files without ROOT, `mplhep` for plottingh, `histoprint` for printing) use the UHI Protocol to communicate histograms.

# Performance example

In [ ]:
# Data
data1 = np.random.normal(3.5, 2.5, size=1_000_000)

In [ ]:
%%timeit
H, e = np.histogram(data1, bins=100, range=(0,5))

In [ ]:
%%timeit
H, e = hist.numpy.histogram(data1, bins=100, range=(0,5))

This is a little unfair - this is "normal" boost-histogrma against an optimized NumPy routine. Let's try an unoptimized one:

In [ ]:
# Second dimension of data
data2 = np.random.normal(3.5, 2.5, size=1_000_000)

In [ ]:
%%timeit
H, e1, e2 = np.histogram2d(data1, data2, bins=(100, 100), range=((0,5), (0, 5)))

In [ ]:
%%timeit
H, e1, e2 = hist.numpy.histogram2d(data1, data2, bins=(100, 100), range=((0,5), (0, 5)))

In [ ]:
%%timeit
H, e1, e2 = hist.numpy.histogram2d(data1, data2, bins=(100, 100), range=((0,5), (0, 5)), threads=4)

# Manipulation example

In [ ]:
sp = Image.open("SciPy Icon.png")

In [ ]:
arr = np.asarray(sp)[:, :, 3]
x, y, p = np.random.default_rng().random((3, 200_000))
bx = (x * 140).astype(np.int32)
by = (y * 120).astype(np.int32)

# Probibility of hit
prob = arr[by, bx] / 255

# True if hit
hits = prob > p

# Make lists of x and y values
X = x[hits]
Y = 1 - (y[hits] / 14 * 12 + (1 / 14));

In [ ]:
h = Hist(
    hist.axis.Regular(300, 0, 1, name="x"),
    hist.axis.Regular(300, 0, 1, name="y"),
)

h.fill(x=X, y=Y);

In [ ]:
h.plot()
plt.gca().set_aspect("equal")

In [ ]:
h[.8j:, .6j:.75j].plot()
plt.gca().set_aspect("equal")

In [ ]:
hx = h.project("x")

In [ ]:
plt.stairs(hx.values(), *hx.axes.edges)

# Profile example

In [ ]:
data = np.vstack(
    [
        np.concatenate(
            [
                np.random.normal(-0.75, 0.3, 1_000_000),
                np.random.normal(0.75, 0.3, 750_000),
                np.random.normal(-1.5, 0.4, 400_000),
            ]
        ),
        np.concatenate(
            [
                np.random.normal(-0.75, 0.8, 1_000_000),
                np.random.normal(0.75, 0.5, 750_000),
                np.random.normal(-1.5, 0.1, 400_000),
            ]
        ),
    ]
);

In [ ]:
h = Hist.new.Reg(100, -3.5, 3.5).Reg(100, -3.5, 3.5).Weight().fill(*data)
h.plot()

In [ ]:
# from matplotlib.colors import LogNorm
fig, ax = plt.subplots(figsize=(8,8))
py = h.profile(1)
h.plot(ax=ax) # , norm=LogNorm(vmin=1, vmax=10_000))
ax.errorbar(*py.axes.centers, py.values(), py.variances()**.5, fmt='r.')
ax.set_aspect("equal")

# Pie plotting example

Also see
https://github.com/pypa/manylinux/issues/994#issuecomment-792210874

In this exmaple, we will read in a pandas example dataset from seaborn-data.

In [ ]:
tips = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv')

In [ ]:
tips.describe()

#### Hist can read from columnar datasets

In [ ]:
h = Hist.from_columns(
    tips,
    [
        hist.axis.Regular(10, 0, 50, name="total_bill"),
        hist.axis.Regular(20, 1, 10, name="tip"),
        "day",
        "size",
    ],
)


#### Let's see the total bill distribution

In [ ]:
h.project("total_bill")

#### How about tips vs. bills?

In [ ]:
htt = h.project("total_bill", "tip").plot2d_full();

#### How about computing the mean over tips at each point? (profile)

In [ ]:
tip = htt.profile("tip")
htt.plot()
plt.errorbar(*tip.axes.centers, tip.values(), tip.variances()**.5, fmt='r.');

#### What about splitting it by day?

In [ ]:
h.project("total_bill", "day").plot()
plt.legend();

#### How about total bill on Saterday only?

In [ ]:
h[:, sum, "Sat", sum]

#### Finally, how about seeing the distribution per day?

In [ ]:
h.project("day").plot_pie(normalize=True, autopct='%1.0f%%', pctdistance=.8);